In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.simplefilter(action='ignore')

## Load dataset

In [2]:
pd.pandas.set_option('display.max_columns', None)
data = pd.read_csv('../data/raw/houseprices.csv')
data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,Gas

## Split dataset into train and test set

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data, data['SalePrice'], test_size=0.1, random_state=0)

print("Train set:", X_train.shape)
print("Test set:", X_test.shape)

Train set: (1314, 81)
Test set: (146, 81)


## Missing values in numerical variables

In [4]:
# Create a list with numerical variables that contain missing value

num_variables_with_nan = []

for variable in data.columns:
    if X_train[variable].isnull().sum() > 0 and X_train[variable].dtypes != 'O':
        num_variables_with_nan.append(variable)

X_train[num_variables_with_nan].isnull().mean()

LotFrontage    0.177321
MasVnrArea     0.004566
GarageYrBlt    0.056317
dtype: float64

In [5]:
# Replace missing values by the mode

for variable in num_variables_with_nan:
    mode_value = X_train[variable].mode()[0]
    X_train[variable] = X_train[variable].fillna(mode_value)
    X_test[variable] = X_test[variable].fillna(mode_value)

X_train[num_variables_with_nan].isnull().sum()

LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

## Missing values in categorical variables

In [6]:
# Create a list with categorical variables that contain missing value

cat_variables_with_nan = []

for variable in data.columns:
    if X_train[variable].isnull().sum() > 0 and X_train[variable].dtypes == 'O':
        cat_variables_with_nan.append(variable)

X_train[cat_variables_with_nan].isnull().mean()

Alley           0.938356
MasVnrType      0.004566
BsmtQual        0.024353
BsmtCond        0.024353
BsmtExposure    0.025114
BsmtFinType1    0.024353
BsmtFinType2    0.025114
Electrical      0.000761
FireplaceQu     0.472603
GarageType      0.056317
GarageFinish    0.056317
GarageQual      0.056317
GarageCond      0.056317
PoolQC          0.995434
Fence           0.814307
MiscFeature     0.961187
dtype: float64

In [7]:
# Replace missing values with new label: "Missing"

X_train[cat_variables_with_nan] = X_train[cat_variables_with_nan].fillna('Missing')
X_test[cat_variables_with_nan] = X_test[cat_variables_with_nan].fillna('Missing')

X_train[cat_variables_with_nan].isnull().sum()

Alley           0
MasVnrType      0
BsmtQual        0
BsmtCond        0
BsmtExposure    0
BsmtFinType1    0
BsmtFinType2    0
Electrical      0
FireplaceQu     0
GarageType      0
GarageFinish    0
GarageQual      0
GarageCond      0
PoolQC          0
Fence           0
MiscFeature     0
dtype: int64

## Temporal variables

In [8]:
# Get the time elapsed between variable and the year in which the house was sold

temporal_variables = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']

for variable in temporal_variables:
    X_train[variable] = X_train['YrSold'] - X_train[variable]
    X_test[variable] = X_test['YrSold'] - X_test[variable]

## Numerical variable transformation

In [10]:
# Log transformation of non-normal distributed variables

non_normal_dist_variables = ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']

for variable in non_normal_dist_variables:
    X_train[variable] = np.log(X_train[variable])
    X_test[variable] = np.log(X_test[variable])

## Rare labels in categorical variables

In [11]:
# Replace rare labels (which appear only in a small proportion of the observations) by the string "Rare"

categorical_variables = []

for variable in X_train.columns:
    if data[variable].dtypes == 'O':
        categorical_variables.append(variable)    

In [12]:
def find_non_rare_labels(df, variable, tolerance):
    df = df.copy()
    temp = df.groupby(variable)['SalePrice'].count() / len(df)
    return temp[temp > tolerance].index

In [13]:
for variable in categorical_variables:
    frequent_category = find_non_rare_labels(X_train, variable, 0.01)
    
    X_train[variable] = np.where(X_train[variable].isin(frequent_category), X_train[variable], 'Rare')
    X_test[variable] = np.where(X_test[variable].isin(frequent_category), X_test[variable], 'Rare')

## Categorical variable encoding

In [14]:
# Transform the strings of the categorical variables into numbers

def replace_categories(train, test, variable, target):
    ordered_labels = train.groupby([variable])[target].mean().sort_values().index

    dict_ordered_labels = {k: i for i, k in enumerate(ordered_labels, 0)}

    train[variable] = train[variable].map(dict_ordered_labels)
    test[variable] = test[variable].map(dict_ordered_labels)

In [15]:
for variable in categorical_variables:
    replace_categories(X_train, X_test, variable, 'SalePrice')

## Feature scaling

In [16]:
# Scaling variables to the minimum and maximum values (MinMaxScaler)

variables_to_scale = [variable for variable in X_train.columns if variable not in ['Id', 'SalePrice']]

scaler = MinMaxScaler()

scaler.fit(X_train[variables_to_scale])

X_train[variables_to_scale] = scaler.transform(X_train[variables_to_scale])
X_test[variables_to_scale] = scaler.transform(X_test[variables_to_scale])

In [17]:
X_train.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
930,931,0.000000,0.75,0.461171,0.377048,1.0,1.0,0.333333,1.000000,1.0,0.0,0.0,0.863636,0.4,1.0,0.75,0.6,0.777778,0.50,0.014706,0.049180,0.0,0.0,1.0,1.0,0.000000,0.00000,0.666667,1.000000,1.0,0.75,0.75,0.75,1.000000,0.002835,0.666667,0.000000,0.673479,0.239935,1.0,1.00,1.0,1.0,0.559760,0.000000,0.0,0.523250,0.000000,0.0,0.666667,0.0,0.375,0.333333,0.666667,0.416667,1.0,0.000000,0.2,0.8,0.018692,1.000000,0.75,0.430183,0.666667,1.0,1.0,0.116686,0.032907,0.0,0.000000,0.000000,0.000000,0.0,0.75,1.0,0.000000,0.545455,0.75,0.666667,0.75,12.211060
656,657,0.000000,0.75,0.456066,0.399443,1.0,1.0,0.333333,0.333333,1.0,0.0,0.0,0.363636,0.4,1.0,0.75,0.6,0.444444,0.75,0.360294,0.049180,0.0,0.0,0.6,0.6,0.666667,0.03375,0.666667,1.000000,0.5,0.50,0.75,0.25,0.666667,0.142807,0.666667,0.000000,0.114724,0.172340,1.0,1.00,1.0,1.0,0.434539,0.000000,0.0,0.406196,0.333333,0.0,0.333333,0.5,0.375,0.333333,0.666667,0.250000,1.0,0.000000,0.2,0.8,0.457944,0.666667,0.25,0.220028,0.666667,1.0,1.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.50,1.0,0.000000,0.636364,0.50,0.666667,0.75,11.887931
45,46,0.588235,0.75,0.394699,0.347082,1.0,1.0,0.000000,0.333333,1.0,0.0,0.0,0.954545,0.4,1.0,1.00,0.6,0.888889,0.50,0.036765,0.098361,1.0,0.0,0.3,0.2,0.666667,0.25750,1.000000,1.000000,1.0,1.00,0.75,0.25,1.000000,0.080794,0.666667,0.000000,0.601951,0.286743,1.0,1.00,1.0,1.0,0.627205,0.000000,0.0,0.586296,0.333333,0.0,0.666667,0.0,0.250,0.333333,1.000000,0.333333,1.0,0.333333,0.8,0.8,0.046729,0.666667,0.50,0.406206,0.666667,1.0,1.0,0.228705,0.149909,0.0,0.000000,0.000000,0.000000,0.0,0.75,1.0,0.000000,0.090909,1.00,0.666667,0.75,12.675764
1348,1349,0.000000,0.75,0.388581,0.493677,1.0,1.0,0.666667,0.666667,1.0,0.0,0.0,0.454545,0.4,1.0,0.75,0.6,0.666667,0.50,0.066176,0.163934,0.0,0.0,1.0,1.0,0.000000,0.00000,0.666667,1.000000,1.0,0.75,0.75,1.00,1.000000,0.255670,0.666667,0.000000,0.018114,0.242553,1.0,1.00,1.0,1.0,0.566920,0.000000,0.0,0.529943,0.333333,0.0,0.666667,0.0,0.375,0.333333,0.666667,0.250000,1.0,0.333333,0.4,0.8,0.084112,0.666667,0.50,0.362482,0.666667,1.0,1.0,0.469078,0.045704,0.0,0.000000,0.000000,0.000000,0.0,0.75,1.0,0.000000,0.636364,0.25,0.666667,0.75,12.278393
55,56,0.000000,0.75,0.577658,0.402702,1.0,1.0,0.333333,0.333333,1.0,0.0,0.0,0.363636,0.4,1.0,0.75,0.6,0.555556,0.50,0.323529,0.737705,0.0,0.0,0.6,0.7,0.666667,0.17000,0.333333,1.000000,0.5,0.50,0.75,0.25,0.333333,0.086818,0.666667,0.000000,0.434278,0.233224,1.0,0.75,1.0,1.0,0.549026,0.000000,0.0,0.513216,0.000000,0.0,0.666667,0.0,0.375,0.333333,0.333333,0.416667,1.0,0.333333,0.8,0.8,0.411215,0.666667,0.50,0.406206,0.666667,1.0,1.0,0.000000,0.000000,0.0,0.801181,0.000000,0.000000,0.0,0.75,1.0,0.000000,0.545455,0.50,0.666667,0.75,12.103486
1228,1229,0.588235,0.75,0.418208,0.373596,1.0,1.0,0.000000,0.333333,1.0,0.5,0.0,0.954545,0.4,1.0,1.00,0.6,0.888889,0.50,0.000000,0.016393,1.0,0.0,0.3,0.2,0.666667,0.47875,1.000000,1.000000,1.0,1.00,0.75,0.25,1.000000,0.272856,0.666667,0.000000,0.075244,0.278560,1.0,1.00,1.0,1.0,0.616248,0.000000,0.0,0.576053,0.333333,0.0,0.333333,0.5,0.125,0.333333,1.000000,0.416667,1.0,0.333333,0.8,0.8,0.000000,1.000000,0.75,0.7

## Saving processed train and test sets

In [19]:
#X_train.to_csv('../data/processed/train_processed.csv', index=False)
#X_test.to_csv('../data/processed/test_processed.csv', index=False)